In [7]:
#loading necessary libraries
import nltk
nltk.download("punkt")
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import numpy as np
#from collections import defaultdict
#from nltk import pos_tag
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
#setting same randomization every time
np.random.seed(500)


In [3]:
init_df=pd.read_csv("/content/train main.xls" , encoding="latin-1", usecols=['category','text'] , delimiter=",")

In [4]:
#cleaning and tokenizing begins
init_df["text"].dropna(inplace=True)
init_df['text']=[entry.lower() for entry in init_df['text']]
init_df['text']= [word_tokenize(entry) for entry in init_df['text']]

In [5]:
#check if the categoris have nearly same number of texts
init_df['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [8]:
lemmatizer= WordNetLemmatizer()
Stemmer = PorterStemmer()
for index, entry in enumerate(init_df['text']) :
    Final_words = []
    for word in entry :
           if word not in stopwords.words('english') and word.isalpha() :
            word_Final = Stemmer.stem(word)
            Final_words.append(word_Final)
           elif not word.isalpha():
                Final_words.append(word)
    init_df.loc[index,'cleaned_text']=str(Final_words)


In [9]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(init_df['cleaned_text'],init_df['category'],test_size=0.2)

In [10]:
#check if the train dataset's categoris have nearly same number of texts
min=Train_Y.value_counts().min()

In [11]:
Tfidf_vect = TfidfVectorizer(min_df=4)
Tfidf_vect.fit(init_df['cleaned_text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [12]:
NB = naive_bayes.MultinomialNB(alpha=1)
NB.fit(Train_X_Tfidf,Train_Y)

# predict the labels on the test dataset
pred_NB = NB.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(pred_NB, Test_Y)*100)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
confmatrix=confusion_matrix(pred_NB, Test_Y)
print (confmatrix)

Naive Bayes Accuracy Score ->  98.42696629213484
[[102   0   2   1   0]
 [  1  76   0   0   0]
 [  0   1  87   0   0]
 [  0   0   0  80   1]
 [  1   0   0   0  93]]


In [ ]:
columns = Tfidf_vect.get_feature_names_out()
train_tfidf_df = pd.DataFrame(Train_X_Tfidf.toarray(), columns=columns)
test_tfidf_df = pd.DataFrame(Test_X_Tfidf.toarray(), columns=columns)

In [22]:
#For testing
Test_main=pd.read_csv('/content/tester.csv',usecols=['text'])    #enter the path to test dataset containing only texts,making sure its header is apt
Test_main['text'].dropna(inplace=True)
Test_main['text']=[entry.lower() for entry in Test_main['text']]

In [23]:

Test_main['tokenized']= [word_tokenize(entry) for entry in Test_main['text']]

In [25]:

for index, entry in enumerate(Test_main['tokenized']) :
    Final_words = []
    for word in entry :
           if word not in stopwords.words('english') and word.isalpha() :
            word_Final = Stemmer.stem(word)
            Final_words.append(word_Final)
           elif not word.isalpha():
                Final_words.append(word)
    Test_main.loc[index,'cleaned_text']=str(Final_words)

In [27]:
Vect=Tfidf_vect.transform(Test_main['cleaned_text'])
pred_NB2 = NB.predict(Vect)